In [1]:
import requests
import keys
import pandas as pd

In [2]:
def fetch(fields, max_per_call=500):
    base_url = 'https://api.igdb.com/v4/games'
    headers = {
        'Client-ID': keys.API_CLIENT_ID,  
        'Authorization': f'Bearer {keys.API_AUTHORIZATION_TOKEN}', 
    }
    
    all_data = []
    offset = 0
    
    while True:
        data = f"fields {','.join(fields)}; limit {max_per_call}; offset {offset}; where first_release_date>631132200 & rating>50;"
        response = requests.post(base_url, headers=headers, data=data)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code}, {response.text}")
            break
        
        games = response.json()  
        
        if not games:
            break
        all_data.extend(games)  
        print(f"Scraped {offset+500} games")
        if len(games) < max_per_call:
            break
        
        offset += max_per_call  
    
    return all_data  


In [3]:
fields = [
    'id', 
    'age_ratings', 
    'artworks', 
    'category', 
    'cover', 
    'dlcs', 
    'first_release_date', 
    'franchise', 
    'game_modes', 
    'genres', 
    'involved_companies', 
    'keywords', 
    'name', 
    'platforms', 
    'player_perspectives', 
    'ports', 
    'release_dates', 
    'remakes', 
    'remasters', 
    'screenshots', 
    'similar_games', 
    'slug', 
    'status', 
    'summary', 
    'total_rating', 
    'url', 
    'videos', 
    'websites'
]



In [4]:
fetched_games = fetch(fields)


Scraped 500 games
Scraped 1000 games
Scraped 1500 games
Scraped 2000 games
Scraped 2500 games
Scraped 3000 games
Scraped 3500 games
Scraped 4000 games
Scraped 4500 games
Scraped 5000 games
Scraped 5500 games
Scraped 6000 games
Scraped 6500 games
Scraped 7000 games
Scraped 7500 games
Scraped 8000 games
Scraped 8500 games
Scraped 9000 games
Scraped 9500 games
Scraped 10000 games
Scraped 10500 games
Scraped 11000 games
Scraped 11500 games
Scraped 12000 games
Scraped 12500 games
Scraped 13000 games
Scraped 13500 games
Scraped 14000 games
Scraped 14500 games
Scraped 15000 games
Scraped 15500 games
Scraped 16000 games
Scraped 16500 games
Scraped 17000 games
Scraped 17500 games
Scraped 18000 games


In [5]:
print(len(fetched_games))
print(fetched_games[10])

17749
{'id': 119719, 'artworks': [34854], 'category': 0, 'cover': 186324, 'first_release_date': 1559865600, 'game_modes': [1], 'genres': [8, 9, 13, 31, 32], 'involved_companies': [102864], 'name': 'Flux Caves', 'platforms': [3, 6], 'player_perspectives': [1], 'release_dates': [201669, 201670], 'screenshots': [315870, 315871, 315872, 315873, 315874, 315875, 315876, 315877, 315878, 315879, 315880, 315881], 'similar_games': [17350, 65827, 75948, 94741, 106992, 111043, 113895, 115850, 119788, 121309], 'slug': 'flux-caves', 'summary': 'Flux Caves is a 3D puzzle game. My plan is to create something which can be a challenge, but still you enjoy to play. I want to create a soothing environment and this is enhanced by slow piano-based music.', 'total_rating': 70.0, 'url': 'https://www.igdb.com/games/flux-caves', 'videos': [37906], 'websites': [110435, 110436, 145455, 145456, 145457, 426195]}


In [6]:

df=pd.DataFrame(fetched_games)

In [7]:
df.head()

,id,age_ratings,artworks,category,cover,dlcs,first_release_date,game_modes,genres,involved_companies,...,summary,total_rating,url,videos,websites,status,ports,franchise,remasters,remakes
0,1443,"[13005, 47239, 47240, 99973, 139045, 139046]","[3014, 3015, 3016, 3017, 3018]",0,288034.0,"[237415, 237416, 237417, 237418, 237419, 23742...",1334793600,[1],"[12, 15]","[3106, 51776, 264417]",...,Armies clash as the world burns. As an inevita...,88.020803,https://www.igdb.com/games/fire-emblem-awakening,"[971, 972]","[10066, 51150, 51151, 334186]",NaN,NaN,NaN,NaN,NaN
1,236547,NaN,NaN,0,310658.0,NaN,1690934400,[1],"[15, 24, 32]",[224218],...,A minimalist game about building and defending...,77.856060,https://www.igdb.com/games/thronefall,"[97151, 118938]","[528578, 556705, 570629]",NaN,NaN,NaN,NaN,NaN
2,1438,[47448],"[110214, 110215, 110216, 110217]",0,309491.0,NaN,1017360000,[1],"[12, 15, 24]","[154367, 154369, 226309, 264446]",...,Fire Emblem: The Binding Blade is a Japanese t...,73.891594,https://www.igdb.com/games/fire-emblem-the-bin...,"[36543, 57291]","[61109, 142038, 142039, 335449]",NaN,NaN,NaN,NaN,NaN
3,10402,NaN,[9037],0,104252.0,NaN,1410998400,[1],"[5, 32]","[83355, 96895]",...,Armed with a simple yet stylish revolver and s...,70.233917,https://www.igdb.com/games/heavy-bullets,[5140],"[44116, 44117, 238156, 397643]",NaN,NaN,NaN,NaN,NaN
4,21863,"[9824, 35875, 47242, 139049, 139050, 139051]",NaN,0,116036.0,NaN,1435190400,"[1, 2]","[12, 15, 16]","[40703, 40704, 131038]",...,Fire Emblem Fates is split into three story pa...,85.549137,https://www.igdb.com/games/fire-emblem-fates-c...,[120485],"[10003, 10004, 240679, 411223]",NaN,NaN,NaN,NaN,NaN


In [8]:
df.to_csv("games.csv",index=False)

In [9]:
games=pd.read_csv("games.csv")

In [10]:
games.head()

,id,age_ratings,artworks,category,cover,dlcs,first_release_date,game_modes,genres,involved_companies,...,summary,total_rating,url,videos,websites,status,ports,franchise,remasters,remakes
0,1443,"[13005, 47239, 47240, 99973, 139045, 139046]","[3014, 3015, 3016, 3017, 3018]",0,288034.0,"[237415, 237416, 237417, 237418, 237419, 23742...",1334793600,[1],"[12, 15]","[3106, 51776, 264417]",...,Armies clash as the world burns. As an inevita...,88.020803,https://www.igdb.com/games/fire-emblem-awakening,"[971, 972]","[10066, 51150, 51151, 334186]",NaN,NaN,NaN,NaN,NaN
1,236547,NaN,NaN,0,310658.0,NaN,1690934400,[1],"[15, 24, 32]",[224218],...,A minimalist game about building and defending...,77.856060,https://www.igdb.com/games/thronefall,"[97151, 118938]","[528578, 556705, 570629]",NaN,NaN,NaN,NaN,NaN
2,1438,[47448],"[110214, 110215, 110216, 110217]",0,309491.0,NaN,1017360000,[1],"[12, 15, 24]","[154367, 154369, 226309, 264446]",...,Fire Emblem: The Binding Blade is a Japanese t...,73.891594,https://www.igdb.com/games/fire-emblem-the-bin...,"[36543, 57291]","[61109, 142038, 142039, 335449]",NaN,NaN,NaN,NaN,NaN
3,10402,NaN,[9037],0,104252.0,NaN,1410998400,[1],"[5, 32]","[83355, 96895]",...,Armed with a simple yet stylish revolver and s...,70.233917,https://www.igdb.com/games/heavy-bullets,[5140],"[44116, 44117, 238156, 397643]",NaN,NaN,NaN,NaN,NaN
4,21863,"[9824, 35875, 47242, 139049, 139050, 139051]",NaN,0,116036.0,NaN,1435190400,"[1, 2]","[12, 15, 16]","[40703, 40704, 131038]",...,Fire Emblem Fates is split into three story pa...,85.549137,https://www.igdb.com/games/fire-emblem-fates-c...,[120485],"[10003, 10004, 240679, 411223]",NaN,NaN,NaN,NaN,NaN


In [11]:
games.shape

(17749, 28)

In [12]:
games.isnull().sum()

id                         0
age_ratings             6618
artworks                6761
category                   0
cover                    542
dlcs                   16388
first_release_date         0
game_modes              1314
genres                   359
involved_companies      2967
keywords                6219
name                       0
platforms                  0
player_perspectives     5984
release_dates              0
screenshots              304
similar_games            187
slug                       0
summary                   44
total_rating               0
url                        0
videos                  6855
websites                 283
status                 16364
ports                  16780
franchise              17080
remasters              17336
remakes                17600
dtype: int64

In [15]:
games.shape

(17749, 28)